In [ ]:
# Bu hücre sadece gerekli kütüphaneleri kurar.
!pip install -q diffusers transformers accelerate torch torchvision Pillow
print("Kütüphaneler kuruldu veya zaten kurulu.")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 96.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 93.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 63.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 35.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 87.0 MB/s eta 0:00:00
Kütüphaneler kuruldu veya zaten kurulu.


In [ ]:
# Bu hücre kütüphaneleri import eder, Drive'ı bağlar, yolları ayarlar
# ve SADECE Kandinsky 2.2 Inpainting modelini yükler.

import torch
# Kandinsky için gerekli pipeline'ı import et
from diffusers import AutoPipelineForInpainting
from diffusers.utils import load_image
from PIL import Image
import os
import json # Metadata okumak için
import time # Zamanlama için
from google.colab import drive
from IPython.display import display, Image as IPImage
import math # Resize fonksiyonu için gerekebilir

# --- Google Drive Bağlantısı ---
try:
    # Daha önce bağlandıysa tekrar sormaz genellikle
    drive.mount('/content/drive', force_remount=False) # force_remount=False ekleyebiliriz
    drive_mounted = True
except Exception as e:
    print(f"!! Google Drive bağlanamadı: {e}. Dosya yolları çalışmayabilir.")
    drive_mounted = False

# --- Temel Ayarlar ---
setup_complete = False # Başlangıçta False, sonunda True olacak
if drive_mounted:
    # LÜTFEN BU YOLU KENDİ GOOGLE DRIVE KLASÖR YAPINIZA GÖRE GÜNCELLEYİN!
    base_project_dir = '/content/drive/MyDrive/Research_Project_ImageEditing/' # Örnek yol, kendinizinkini yazın
    print(f"Google Drive bağlandı. Proje kök dizini: {base_project_dir}")

    image_dir = os.path.join(base_project_dir, "Images")
    mask_dir = os.path.join(base_project_dir, "Masks")
    metadata_path = os.path.join(base_project_dir, "metadata.json")

    # Çıktıları Kandinsky için ayrı bir klasöre kaydedelim
    output_dir_kandinsky = os.path.join(base_project_dir, "Outputs", "KandinskyInpainting")

    device = "cuda" if torch.cuda.is_available() else "cpu"

    # Gerekli klasörleri oluştur (varsa hata verme)
    os.makedirs(output_dir_kandinsky, exist_ok=True)
    if not os.path.isdir(image_dir): print(f"!! UYARI: Görüntü klasörü bulunamadı: {image_dir}")
    if not os.path.isdir(mask_dir): print(f"!! UYARI: Maske klasörü bulunamadı: {mask_dir}")
    if not os.path.isfile(metadata_path): print(f"!! UYARI: Metadata dosyası bulunamadı: {metadata_path}")

    print(f"Kullanılan cihaz: {device}")
    print(f"Girdi Görüntü klasörü: {image_dir}")
    print(f"Girdi Maske klasörü: {mask_dir}")
    print(f"Metadata dosyası: {metadata_path}")
    print(f"Çıktı klasörü (Kandinsky): {output_dir_kandinsky}")

    # --- Model Ayarları (Kandinsky 2.2 Inpainting) ---
    # Kandinsky 2.2 genellikle 768 veya 1024'te iyi çalışır. 768'i seçelim.
    kandinsky_target_resolution = 768
    print(f"\nKandinsky Hedef Çözünürlük (En Uzun Kenar): {kandinsky_target_resolution}px")

    # Model Parametreleri (Ayarlanabilir - Başlangıç değerleri)
    kandinsky_inference_steps = 50  # Kandinsky için de makul bir başlangıç
    # Kandinsky genellikle daha düşük guidance scale ile iyi çalışır
    kandinsky_guidance_scale = 4.0
    print(f"Model Parametreleri: steps={kandinsky_inference_steps}, guidance_scale={kandinsky_guidance_scale}")

    setup_complete = True # Ayarlar tamamlandı

    # --- Model Yükleme (Kandinsky 2.2 Inpainting) ---
    # Kandinsky'nin Prior ve Inpainting pipeline'larına ihtiyacı var.
    if setup_complete:
        print("\nKandinsky 2.2 Inpainting modeli yükleniyor (Prior ve Decoder)...")
        try:
            # 1. Prior Pipeline (Text'ten image embedding üretir)
            print("- Prior pipeline yükleniyor...")
            pipe_kandinsky_inpaint = AutoPipelineForInpainting.from_pretrained("kandinsky-community/kandinsky-2-1-inpaint", torch_dtype=torch.float16).to(device)
            pipe_kandinsky_inpaint.enable_model_cpu_offload()
            print("- Prior pipeline yüklendi.")


            # İki pipeline da başarıyla yüklendiyse model hazır
            model_loaded = True
            print("- Kandinsky 2.2 Inpainting modeli başarıyla yüklendi.")

        except Exception as e:
            print(f"!! Kandinsky yüklenirken HATA: {e}")
            print("!! Bellek hatası alıyorsanız, Colab Runtime'ı yeniden başlatıp SADECE BU notebook'u çalıştırmayı deneyin.")
            prior_pipe = None
            pipe_kandinsky_inpaint = None
            model_loaded = False
    else:
        print("!! Temel ayarlar tamamlanamadığı için model yüklenemiyor.")
        model_loaded = False

else:
    print("!! Drive bağlanmadığı için devam edilemiyor.")
    model_loaded = False
    setup_complete = False

# Son durumu kontrol etmek için küçük bir özet
if setup_complete and model_loaded:
    print("\n--- Kurulum Tamamlandı ve Kandinsky Modeli Hazır ---")
elif setup_complete and not model_loaded:
    print("\n--- Kurulum Ayarları Tamamlandı ama Model Yüklenemedi ---")
else:
    print("\n--- Kurulum Tamamlanamadı ---")

Mounted at /content/drive
Google Drive bağlandı. Proje kök dizini: /content/drive/MyDrive/SNU_Project_ImageEditing/
Kullanılan cihaz: cuda
Girdi Görüntü klasörü: /content/drive/MyDrive/SNU_Project_ImageEditing/Images
Girdi Maske klasörü: /content/drive/MyDrive/SNU_Project_ImageEditing/Masks
Metadata dosyası: /content/drive/MyDrive/SNU_Project_ImageEditing/metadata.json
Çıktı klasörü (Kandinsky): /content/drive/MyDrive/SNU_Project_ImageEditing/Outputs/KandinskyInpainting

Kandinsky Hedef Çözünürlük (En Uzun Kenar): 768px
Model Parametreleri: steps=50, guidance_scale=4.0

Kandinsky 2.2 Inpainting modeli yükleniyor (Prior ve Decoder)...
- Prior pipeline yükleniyor...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model_index.json:   0%|          | 0.00/383 [00:00<?, ?B/s]

Fetching 13 files:   0%|          | 0/13 [00:00<?, ?it/s]

config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/460 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.06M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

- Prior pipeline yüklendi.
- Kandinsky 2.2 Inpainting modeli başarıyla yüklendi.

--- Kurulum Tamamlandı ve Kandinsky Modeli Hazır ---


In [ ]:
# GEREKLİYSE: resize_image_multiple_of_64 fonksiyonunu burada tanımla
# Eğer zaten başka bir hücrede tanımlıysa veya import edildiyse bu adıma gerek yok.

from PIL import Image
import math

def resize_image_multiple_of_64(image, longest_side):
    """
    Bir görüntüyü, en uzun kenarı belirtilen 'longest_side' değerine eşit veya daha küçük olacak
    şekilde yeniden boyutlandırır, en boy oranını korur ve SONUÇ boyutlarının (genişlik ve yükseklik)
    64'ün katları olmasını sağlar.

    Args:
        image (PIL.Image.Image): Yeniden boyutlandırılacak PIL Görüntü nesnesi.
        longest_side (int): En uzun kenarın hedef maksimum piksel değeri.

    Returns:
        PIL.Image.Image: Yeniden boyutlandırılmış PIL Görüntü nesnesi.
        tuple: (final_w, final_h) son boyutları içeren tuple.
    """
    if not isinstance(image, Image.Image):
        raise TypeError("Girdi 'image' bir PIL Image nesnesi olmalıdır.")

    width, height = image.size
    original_size_str = f"{width}x{height}"

    # 1. Adım: Hedef boyuta göre ilk ölçeklemeyi hesapla
    if max(width, height) <= longest_side:
        target_w, target_h = width, height
    else:
        if width > height:
            ratio = longest_side / width
            target_w = longest_side
            target_h = int(height * ratio)
        else:
            ratio = longest_side / height
            target_h = longest_side
            target_w = int(width * ratio)

    # 2. Adım: Hesaplanan hedef boyutları 64'ün en yakın katına yuvarla
    final_w = max(64, round(target_w / 64) * 64)
    final_h = max(64, round(target_h / 64) * 64)

    final_size_str = f"{final_w}x{final_h}"

    # 3. Adım: Eğer son boyutlar orijinal boyutlarla aynıysa, gereksiz yere yeniden boyutlandırma yapma
    if final_w == width and final_h == height:
        return image, (width, height)

    # 4. Adım: Görüntüyü son, 64'ün katı olan boyutlara yeniden boyutlandır
    try:
        resized_image = image.resize((final_w, final_h), Image.Resampling.LANCZOS)
        return resized_image, (final_w, final_h)
    except Exception as e:
        print(f"!! Görüntü yeniden boyutlandırma sırasında HATA (Orijinal: {original_size_str}, Hedef: {final_size_str}): {e}")
        raise

In [ ]:
# Bu hücre metadata.json dosyasını okur ve tüm girdileri Kandinsky 2.2 Inpainting ile işler.

import json
import time
from PIL import Image
import os
import torch

# Gerekli değişkenlerin önceki hücrelerden geldiğini varsayıyoruz:
# model_loaded, setup_complete, prior_pipe, pipe_kandinsky_inpaint, metadata_path,
# image_dir, mask_dir, output_dir_kandinsky, kandinsky_target_resolution,
# kandinsky_inference_steps, kandinsky_guidance_scale, device

if model_loaded and setup_complete:
    print("\n--- Kandinsky 2.2 Inpainting Toplu İşlem Başlatılıyor ---")

    # Metadata dosyasını yükle
    try:
        with open(metadata_path, 'r', encoding='utf-8') as f:
            metadata = json.load(f)
        print(f"- Metadata dosyası başarıyla okundu ({len(metadata)} girdi bulundu).")
    except Exception as e:
        print(f"!! Metadata dosyası okunamadı: {e}")
        metadata = []

    failed_entries = []
    processed_count = 0
    total_entries = len(metadata)
    loop_start_time = time.time()

    for i, entry in enumerate(metadata):
        entry_id = entry.get("entry_id", f"unknown_entry_{i+1}")
        print(f"\n[{i+1}/{total_entries}] İşleniyor: {entry_id}")
        entry_start_time = time.time()

        try:
            # Gerekli bilgileri JSON'dan al
            input_rel_path = entry.get("input_image_path")
            mask_rel_path = entry.get("mask_path")
            # Kandinsky için de SD ile aynı prompt'u kullanalım
            prompt = entry.get("prompt_sd_k")
            negative_prompt = entry.get("negative_prompt", "")

            if not input_rel_path or not prompt:
                raise ValueError(f"JSON girdisinde 'input_image_path' ({input_rel_path}) veya 'prompt_sd_k' ({prompt}) eksik.")

            input_abs_path = os.path.join(base_project_dir, input_rel_path)

            mask_abs_path = None
            if mask_rel_path:
                mask_abs_path = os.path.join(base_project_dir, mask_rel_path)

            # Çıktı dosya adını ve yolunu oluştur
            output_filename = f"{entry_id}_Kandinsky_{kandinsky_target_resolution}px_output.png"
            output_abs_path = os.path.join(output_dir_kandinsky, output_filename)

            # # Eğer çıktı zaten varsa atla (isteğe bağlı)
            # if os.path.exists(output_abs_path):
            #     print(f" - Çıktı dosyası zaten var, atlanıyor: {output_filename}")
            #     processed_count += 1
            #     # if device == 'cuda': torch.cuda.empty_cache()
            #     continue

            # 1. Girdi resmini yükle
            if not os.path.exists(input_abs_path):
                raise FileNotFoundError(f"Girdi resmi bulunamadı: {input_abs_path}")
            init_image = Image.open(input_abs_path).convert("RGB")
            print(f" - Girdi resmi yüklendi: {input_rel_path} (Orijinal Boyut: {init_image.size})")

            # 2. Görüntüyü hedef çözünürlüğe indir (64'ün katı)
            resized_image, final_dims = resize_image_multiple_of_64(init_image, kandinsky_target_resolution)
            print(f" - Girdi resmi yeniden boyutlandırıldı: {final_dims}")

            # 3. Maskeyi yükle veya oluştur
            resized_mask = None # Başlangıçta None
            if mask_abs_path and os.path.exists(mask_abs_path):
                mask_image_original = Image.open(mask_abs_path).convert("L")
                print(f" - Maske dosyası yüklendi: {mask_rel_path} (Orijinal Boyut: {mask_image_original.size})")
                resized_mask_temp, mask_final_dims = resize_image_multiple_of_64(mask_image_original, kandinsky_target_resolution)

                if final_dims != mask_final_dims:
                     print(f" - UYARI: Yeniden boyutlandırılmış resim ({final_dims}) ve maske ({mask_final_dims}) boyutları farklı! Maske, resim boyutuna zorlanıyor.")
                     resized_mask = resized_mask_temp.resize(final_dims, Image.Resampling.NEAREST)
                     print(f" - Maske son boyuta ({final_dims}) yeniden boyutlandırıldı (NEAREST).")
                else:
                     resized_mask = resized_mask_temp # Boyutlar eşleşiyorsa direkt kullan
                     print(f" - Maske yeniden boyutlandırıldı: {mask_final_dims}")

            elif mask_rel_path is None:
                print(f" - 'mask_path' null. Global değişiklik için beyaz maske ({final_dims}) oluşturuluyor.")
                resized_mask = Image.new("L", final_dims, 255)
            else:
                 raise FileNotFoundError(f"Maske dosyası belirtilmiş ancak bulunamadı: {mask_abs_path}")

            if resized_mask is None:
                raise ValueError("Maske yüklenemedi veya oluşturulamadı.")

            # 4. Modeli Çalıştır (Kandinsky'nin iki aşamalı süreci)
            print(f" - Model çalıştırılıyor (Prompt: '{prompt[:60]}...')")
            model_run_start_time = time.time()

            # Kandinsky 2.1 tarzı doğrudan inpainting kullanımı
            result = pipe_kandinsky_inpaint(
                prompt=prompt,
                negative_prompt=negative_prompt,
                image=resized_image,
                mask_image=resized_mask,
                num_inference_steps=kandinsky_inference_steps,
                guidance_scale=kandinsky_guidance_scale,
                height=final_dims[1],
                width=final_dims[0]
            )

            edited_image = result.images[0]

            model_run_end_time = time.time()
            print(f" - Model çalışması tamamlandı ({model_run_end_time - model_run_start_time:.2f} saniye).")

            # 5. Çıktıyı kaydet
            edited_image.save(output_abs_path)
            print(f" - Çıktı kaydedildi: {output_filename}")
            processed_count += 1

        except Exception as e:
            error_message = f"HATA ({entry_id}): {type(e).__name__} - {e}"
            import traceback
            traceback.print_exc()
            print(f"!! {error_message}")
            failed_entries.append({"entry_id": entry_id, "error": error_message})

        finally:
            if device == 'cuda':
                torch.cuda.empty_cache()

        entry_end_time = time.time()
        print(f" - Girdi {entry_id} tamamlandı ({entry_end_time - entry_start_time:.2f} saniye).")

    loop_end_time = time.time()
    print(f"\n--- Kandinsky 2.2 Inpainting Toplu İşlem Tamamlandı ---")
    print(f"Toplam Süre: {(loop_end_time - loop_start_time)/60:.2f} dakika")
    print(f"Başarıyla İşlenen Girdi Sayısı: {processed_count}/{total_entries}")

    # Hataları Raporla
    if failed_entries:
        print("\n--- Hata Alan Girdiler ---")
        for fail in failed_entries:
            print(f"- ID: {fail['entry_id']}, Hata: {fail['error']}")
    else:
        print("\n- Tüm girdiler hatasız işlendi.")

else:
    print("!! Model yüklenemediği veya kurulum tamamlanmadığı için toplu işlem başlatılamıyor.")


--- Kandinsky 2.2 Inpainting Toplu İşlem Başlatılıyor ---
- Metadata dosyası başarıyla okundu (96 girdi bulundu).

[1/96] İşleniyor: Portrait_01_WomanHatAdd
 - Girdi resmi yüklendi: Images/Portrait_01_WomanHatAdd.png (Orijinal Boyut: (3744, 5616))
 - Girdi resmi yeniden boyutlandırıldı: (512, 768)
 - Maske dosyası yüklendi: Masks/Portrait_01_WomanHatAdd_mask.png (Orijinal Boyut: (3744, 5616))
 - Maske yeniden boyutlandırıldı: (512, 768)
 - Model çalıştırılıyor (Prompt: 'Woman wearing a stylish summer hat, outdoor portrait...')


  0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

 - Model çalışması tamamlandı (9.63 saniye).
 - Çıktı kaydedildi: Portrait_01_WomanHatAdd_Kandinsky_768px_output.png
 - Girdi Portrait_01_WomanHatAdd tamamlandı (10.96 saniye).

[2/96] İşleniyor: Portrait_02_ManNecklaceAdd
 - Girdi resmi yüklendi: Images/Portrait_02_ManNecklaceAdd.png (Orijinal Boyut: (3149, 4724))
 - Girdi resmi yeniden boyutlandırıldı: (512, 768)
 - Maske dosyası yüklendi: Masks/Portrait_02_ManNecklaceAdd_mask.png (Orijinal Boyut: (3149, 4724))
 - Maske yeniden boyutlandırıldı: (512, 768)
 - Model çalıştırılıyor (Prompt: 'Man wearing a silver chain necklace, t-shirt...')


  0%|          | 0/25 [00:00<?, ?it/s]

...

  0%|          | 0/50 [00:00<?, ?it/s]

 - Model çalışması tamamlandı (10.22 saniye).
 - Çıktı kaydedildi: Art_15_MakeDrawingLookOld_Kandinsky_768px_output.png
 - Girdi Art_15_MakeDrawingLookOld tamamlandı (10.69 saniye).

[96/96] İşleniyor: Art_16_AddDetailsToSketch
 - Girdi resmi yüklendi: Images/Art_16_AddDetailsToSketch.png (Orijinal Boyut: (3738, 2334))
 - Girdi resmi yeniden boyutlandırıldı: (768, 448)
 - 'mask_path' null. Global değişiklik için beyaz maske ((768, 448)) oluşturuluyor.
 - Model çalıştırılıyor (Prompt: 'Detailed sketch with intricate shading and texture, finished...')


  0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

 - Model çalışması tamamlandı (9.31 saniye).
 - Çıktı kaydedildi: Art_16_AddDetailsToSketch_Kandinsky_768px_output.png
 - Girdi Art_16_AddDetailsToSketch tamamlandı (9.60 saniye).

--- Kandinsky 2.2 Inpainting Toplu İşlem Tamamlandı ---
Toplam Süre: 16.00 dakika
Başarıyla İşlenen Girdi Sayısı: 96/96

- Tüm girdiler hatasız işlendi.


In [ ]:
selected_entries = [
    "Portrait_11_ToComicStyle",
    "Portrait_12_ToOilPainting"
]

filtered_metadata = []
with open(metadata_path, 'r', encoding='utf-8') as f:
    all_metadata = json.load(f)
    filtered_metadata = [entry for entry in all_metadata if entry.get("entry_id") in selected_entries]

print(f"\n--- Kandinsky 2.2: Seçili Girdiler İşleniyor ({len(filtered_metadata)} giriş) ---")

# Aşağıdaki kod bloğu, zaten yukarıda verdiğin model yürütme döngüsü ile aynı.
# Sadece 'metadata' yerine 'filtered_metadata' kullan.
# Yani döngü şu şekilde başlasın:

for i, entry in enumerate(filtered_metadata):
    entry_id = entry.get("entry_id", f"unknown_entry_{i+1}")
    entry_start_time = time.time()
    try:
        # Gerekli bilgileri JSON'dan al
        input_rel_path = entry.get("input_image_path")
        mask_rel_path = entry.get("mask_path")
        # Kandinsky için de SD ile aynı prompt'u kullanalım
        prompt = entry.get("prompt_sd_k")
        negative_prompt = entry.get("negative_prompt", "")
        if not input_rel_path or not prompt:
            raise ValueError(f"JSON girdisinde 'input_image_path' ({input_rel_path}) veya 'prompt_sd_k' ({prompt}) eksik.")
        input_abs_path = os.path.join(base_project_dir, input_rel_path)
        mask_abs_path = None
        if mask_rel_path:
            mask_abs_path = os.path.join(base_project_dir, mask_rel_path)
        # Çıktı dosya adını ve yolunu oluştur
        output_filename = f"{entry_id}_Kandinsky_{kandinsky_target_resolution}px_output.png"
        output_abs_path = os.path.join(output_dir_kandinsky, output_filename)
        # # Eğer çıktı zaten varsa atla (isteğe bağlı)
        # if os.path.exists(output_abs_path):
        #     print(f" - Çıktı dosyası zaten var, atlanıyor: {output_filename}")
        #     processed_count += 1
        #     # if device == 'cuda': torch.cuda.empty_cache()
        #     continue
        # 1. Girdi resmini yükle
        if not os.path.exists(input_abs_path):
            raise FileNotFoundError(f"Girdi resmi bulunamadı: {input_abs_path}")
        init_image = Image.open(input_abs_path).convert("RGB")
        print(f" - Girdi resmi yüklendi: {input_rel_path} (Orijinal Boyut: {init_image.size})")
        # 2. Görüntüyü hedef çözünürlüğe indir (64'ün katı)
        resized_image, final_dims = resize_image_multiple_of_64(init_image, kandinsky_target_resolution)
        print(f" - Girdi resmi yeniden boyutlandırıldı: {final_dims}")
        # 3. Maskeyi yükle veya oluştur
        resized_mask = None # Başlangıçta None
        if mask_abs_path and os.path.exists(mask_abs_path):
            mask_image_original = Image.open(mask_abs_path).convert("L")
            print(f" - Maske dosyası yüklendi: {mask_rel_path} (Orijinal Boyut: {mask_image_original.size})")
            resized_mask_temp, mask_final_dims = resize_image_multiple_of_64(mask_image_original, kandinsky_target_resolution)
            if final_dims != mask_final_dims:
                 print(f" - UYARI: Yeniden boyutlandırılmış resim ({final_dims}) ve maske ({mask_final_dims}) boyutları farklı! Maske, resim boyutuna zorlanıyor.")
                 resized_mask = resized_mask_temp.resize(final_dims, Image.Resampling.NEAREST)
                 print(f" - Maske son boyuta ({final_dims}) yeniden boyutlandırıldı (NEAREST).")
            else:
                 resized_mask = resized_mask_temp # Boyutlar eşleşiyorsa direkt kullan
                 print(f" - Maske yeniden boyutlandırıldı: {mask_final_dims}")
        elif mask_rel_path is None:
            print(f" - 'mask_path' null. Global değişiklik için beyaz maske ({final_dims}) oluşturuluyor.")
            resized_mask = Image.new("L", final_dims, 255)
        else:
             raise FileNotFoundError(f"Maske dosyası belirtilmiş ancak bulunamadı: {mask_abs_path}")
        if resized_mask is None:
            raise ValueError("Maske yüklenemedi veya oluşturulamadı.")
        # 4. Modeli Çalıştır (Kandinsky'nin iki aşamalı süreci)
        print(f" - Model çalıştırılıyor (Prompt: '{prompt[:60]}...')")
        model_run_start_time = time.time()
        # Kandinsky 2.1 tarzı doğrudan inpainting kullanımı
        result = pipe_kandinsky_inpaint(
            prompt=prompt,
            negative_prompt=negative_prompt,
            image=resized_image,
            mask_image=resized_mask,
            num_inference_steps=kandinsky_inference_steps,
            guidance_scale=kandinsky_guidance_scale,
            height=final_dims[1],
            width=final_dims[0]
        )
        edited_image = result.images[0]
        model_run_end_time = time.time()
        print(f" - Model çalışması tamamlandı ({model_run_end_time - model_run_start_time:.2f} saniye).")
        # 5. Çıktıyı kaydet
        edited_image.save(output_abs_path)
        print(f" - Çıktı kaydedildi: {output_filename}")
    except Exception as e:
        error_message = f"HATA ({entry_id}): {type(e).__name__} - {e}"
        import traceback
        traceback.print_exc()
        print(f"!! {error_message}")
    finally:
        if device == 'cuda':
            torch.cuda.empty_cache()
    entry_end_time = time.time()
    print(f" - Girdi {entry_id} tamamlandı ({entry_end_time - entry_start_time:.2f} saniye).")



--- Kandinsky 2.2: Seçili Girdiler İşleniyor (2 giriş) ---
 - Girdi resmi yüklendi: Images/Portrait_11_ToComicStyle.png (Orijinal Boyut: (4912, 7360))
 - Girdi resmi yeniden boyutlandırıldı: (512, 768)
 - 'mask_path' null. Global değişiklik için beyaz maske ((512, 768)) oluşturuluyor.
 - Model çalıştırılıyor (Prompt: 'Portrait in the style of a comic book art, bold lines, vibra...')


  0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

 - Model çalışması tamamlandı (10.85 saniye).
 - Çıktı kaydedildi: Portrait_11_ToComicStyle_Kandinsky_768px_output.png
 - Girdi Portrait_11_ToComicStyle tamamlandı (14.61 saniye).
 - Girdi resmi yüklendi: Images/Portrait_12_ToOilPainting.png (Orijinal Boyut: (3000, 4499))
 - Girdi resmi yeniden boyutlandırıldı: (512, 768)
 - 'mask_path' null. Global değişiklik için beyaz maske ((512, 768)) oluşturuluyor.
 - Model çalıştırılıyor (Prompt: 'Oil painting portrait, visible brushstrokes...')


  0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

 - Model çalışması tamamlandı (9.59 saniye).
 - Çıktı kaydedildi: Portrait_12_ToOilPainting_Kandinsky_768px_output.png
 - Girdi Portrait_12_ToOilPainting tamamlandı (11.97 saniye).
